In [108]:
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets
import pandas as pd
from datetime import timedelta
import re

In [109]:
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(":"))
    return minutes * 60 + seconds

In [110]:
team_1_roster_df = pd.read_csv('cle_roster_2019.csv')
team_1_roster_list=team_1_roster_df['Roster'].tolist()
print(team_1_roster_list)

['Eli Ankou', 'Genard Avery', 'Odell Beckham Jr.', 'Joel Bitonio', 'Pharaoh Brown', 'Brandin Bryant', 'Morgan Burnett', 'Juston Burris', 'Antonio Callaway', 'Stephen Carlson', 'T.J. Carrie', 'Nick Chubb', 'Austin Corbett', 'Bryan Cox Jr.', 'Tae Davis', 'Daniel Ekuale', 'Drew Forbes', 'Myles Garrett', 'Garrett Gilbert', 'Jamie Gillan', 'Porter Gustin', 'Demetrius Harris', 'Willie Harvey', 'J.T. Hassell', 'Rashard Higgins', 'Dontrell Hilliard', 'KhaDarel Hodge', 'Chris Hubbard', 'Charley Hughlett', 'Kareem Hunt', 'Robert Jackson', 'Malik Jefferson', "D'Ernest Johnson", 'Christian Kirksey', 'Eric Kush', 'Kendall Lamm', 'Jarvis Landry', 'Devaroe Lawrence', 'Baker Mayfield', 'Justin McCray', 'Rob McCray', 'Terrance Mitchell', 'Eric Murray', 'David Njoku', 'Larry Ogunjobi', 'Damarious Randall', 'Damion Ratley', 'Sheldrick Redwine', 'Sheldon Richardson', 'Greg Robinson', 'Joe Schobert', 'Ricky Seals-Jones', 'Austin Seibert', 'Chris Smith', 'Sione Takitaki', 'Adarius Taylor', 'Taywan Taylor', 

In [111]:
cle_bal_2019_df = pd.read_csv('cle_bal_2019.csv')
cle_bal_2019_df.head()

,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail,EPB,EPA
0,1.0,15:00,NaN,NaN,BAL 35,0.0,0.0,"Justin Tucker kicks off 65 yards, touchback.",0.00,0.61
1,1.0,15:00,1.0,10.0,CLE 25,0.0,0.0,Baker Mayfield pass incomplete short middle in...,0.61,0.06
2,1.0,14:57,2.0,10.0,CLE 25,0.0,0.0,Nick Chubb left tackle for 4 yards (tackle by ...,0.06,-0.10
3,1.0,14:15,3.0,6.0,CLE 29,0.0,0.0,Baker Mayfield sacked by Tyus Bowser for -5 yards,-0.10,-1.77
4,1.0,13:39,4.0,11.0,CLE 24,0.0,0.0,Jamie Gillan punts 41 yards out of bounds,-1.77,-1.27


In [112]:
cle_bal_2019_df =cle_bal_2019_df.dropna()
cle_bal_2019_df =cle_bal_2019_df.drop(columns=['EPA','EPB'])
cle_bal_2019_df = cle_bal_2019_df.astype({'Quarter': 'int8', 'Down': 'int8','ToGo': 'int8','Down': 'int8','CLE': 'int8','BAL': 'int8','Time':'string'})
cle_bal_2019_df_clean=cle_bal_2019_df.copy()
cle_bal_2019_df_clean.head()

,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail
1,1,15:00,1,10,CLE 25,0,0,Baker Mayfield pass incomplete short middle in...
2,1,14:57,2,10,CLE 25,0,0,Nick Chubb left tackle for 4 yards (tackle by ...
3,1,14:15,3,6,CLE 29,0,0,Baker Mayfield sacked by Tyus Bowser for -5 yards
4,1,13:39,4,11,CLE 24,0,0,Jamie Gillan punts 41 yards out of bounds
5,1,13:32,1,10,BAL 35,0,0,Mark Ingram right tackle for 2 yards (tackle b...


In [113]:
cle_bal_2019_df_clean['possession_team'] = cle_bal_2019_df['Detail'].apply(lambda x: possession_check(x, team_1_roster_list, "CLE","BAL"))
cle_bal_2019_df_clean = cle_bal_2019_df_clean[cle_bal_2019_df_clean['possession_team'] == 'CLE']
cle_bal_2019_df_clean = cle_bal_2019_df_clean[cle_bal_2019_df_clean['Down'] != 4]
cle_off_df = cle_bal_2019_df_clean.copy()
cle_off_df.head()


,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail,possession_team
1,1,15:00,1,10,CLE 25,0,0,Baker Mayfield pass incomplete short middle in...,CLE
2,1,14:57,2,10,CLE 25,0,0,Nick Chubb left tackle for 4 yards (tackle by ...,CLE
3,1,14:15,3,6,CLE 29,0,0,Baker Mayfield sacked by Tyus Bowser for -5 yards,CLE
9,1,11:21,1,10,CLE 16,0,0,Baker Mayfield pass complete short middle to J...,CLE
10,1,10:44,1,10,CLE 32,0,0,Baker Mayfield pass incomplete deep left inten...,CLE


In [114]:
# Set initial value for the first row
cle_off_df.at[cle_off_df.index[0], 'yardage_diff_1_2'] = cle_off_df.at[cle_off_df.index[0], 'ToGo']
cle_off_df.at[cle_off_df.index[0], 'yardage_diff_2_3'] = 0

# Iterate through the DataFrame to calculate 'yardage_diff_1_2'
for index, row in cle_off_df.iterrows():
    if row['Down'] == 1:  # If Down is 1, set 'yardage_diff_1_2' to "yards to go"
        cle_off_df.at[index, 'yardage_diff_1_2'] = row['ToGo']
    elif row['Down'] == 2:
        if index - 1 in cle_off_df.index:
            cle_off_df.at[index, 'yardage_diff_1_2'] = abs(row['ToGo'] - cle_off_df.at[index - 1, 'ToGo'])
        else:
            cle_off_df.at[index, 'yardage_diff_1_2'] = 0
    elif row['Down'] == 3:
        cle_off_df.at[index, 'yardage_diff_1_2'] = 0

# Set the value of 'yardage_diff_2_3' for all rows (since it remains constant)
cle_off_df['yardage_diff_2_3'] = 0

cle_off_df.head()


,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail,possession_team,yardage_diff_1_2,yardage_diff_2_3
1,1,15:00,1,10,CLE 25,0,0,Baker Mayfield pass incomplete short middle in...,CLE,10.0,0
2,1,14:57,2,10,CLE 25,0,0,Nick Chubb left tackle for 4 yards (tackle by ...,CLE,0.0,0
3,1,14:15,3,6,CLE 29,0,0,Baker Mayfield sacked by Tyus Bowser for -5 yards,CLE,0.0,0
9,1,11:21,1,10,CLE 16,0,0,Baker Mayfield pass complete short middle to J...,CLE,10.0,0
10,1,10:44,1,10,CLE 32,0,0,Baker Mayfield pass incomplete deep left inten...,CLE,10.0,0


In [115]:
for index, row in cle_off_df.iterrows():
    if index > 2 and row['Down'] == 3:  # Exclude the first two rows and only calculate for Down == 3
        if (index - 1) in cle_off_df.index:  # Check if the previous index exists
            cle_off_df.at[index, 'yardage_diff_2_3'] = cle_off_df.at[index - 1, 'ToGo'] - row['ToGo']
        else:
            cle_off_df.at[index, 'yardage_diff_2_3'] = 0  # Set to 0 if the previous index doesn't exist
    else:
        cle_off_df.at[index, 'yardage_diff_2_3'] = 0  # Set to 0 for other rows or when Down != 3
cle_off_df.head()

,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail,possession_team,yardage_diff_1_2,yardage_diff_2_3
1,1,15:00,1,10,CLE 25,0,0,Baker Mayfield pass incomplete short middle in...,CLE,10.0,0
2,1,14:57,2,10,CLE 25,0,0,Nick Chubb left tackle for 4 yards (tackle by ...,CLE,0.0,0
3,1,14:15,3,6,CLE 29,0,0,Baker Mayfield sacked by Tyus Bowser for -5 yards,CLE,0.0,4
9,1,11:21,1,10,CLE 16,0,0,Baker Mayfield pass complete short middle to J...,CLE,10.0,0
10,1,10:44,1,10,CLE 32,0,0,Baker Mayfield pass incomplete deep left inten...,CLE,10.0,0


In [116]:
cle_off_df['sacked'] = 0  # Initialize the 'sacked' column
cle_off_df['sack_yards'] = 0  # Initialize the 'sack yards' column

for index, row in cle_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\b(sacked)\b', play_detail):  # Check for 'sacked' in the play detail
        cle_off_df.at[index, 'sacked'] = 1  # Set 'sacked' to 1 if 'sacked' is found
        # Extract sack yards using regex
        sack_yards_match = re.search(r'(-?\d+)\s+yards', play_detail)  # Match negative or positive numbers
        sack_yards = int(sack_yards_match.group(1))  # Extract the sack yards value
        cle_off_df.at[index, 'sack_yards'] = sack_yards  # Assign sack yards to the corresponding row

cle_off_df.head()


,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail,possession_team,yardage_diff_1_2,yardage_diff_2_3,sacked,sack_yards
1,1,15:00,1,10,CLE 25,0,0,Baker Mayfield pass incomplete short middle in...,CLE,10.0,0,0,0
2,1,14:57,2,10,CLE 25,0,0,Nick Chubb left tackle for 4 yards (tackle by ...,CLE,0.0,0,0,0
3,1,14:15,3,6,CLE 29,0,0,Baker Mayfield sacked by Tyus Bowser for -5 yards,CLE,0.0,4,1,-5
9,1,11:21,1,10,CLE 16,0,0,Baker Mayfield pass complete short middle to J...,CLE,10.0,0,0,0
10,1,10:44,1,10,CLE 32,0,0,Baker Mayfield pass incomplete deep left inten...,CLE,10.0,0,0,0


In [117]:
cle_bal_2019_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 1 to 172
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Quarter          64 non-null     int8  
 1   Time             64 non-null     string
 2   Down             64 non-null     int8  
 3   ToGo             64 non-null     int8  
 4   Location         64 non-null     object
 5   CLE              64 non-null     int8  
 6   BAL              64 non-null     int8  
 7   Detail           64 non-null     object
 8   possession_team  64 non-null     object
dtypes: int8(5), object(3), string(1)
memory usage: 4.9+ KB


for index, row in cle_bal_2019_df_clean.iterrows():
    if index == 0:  
        cle_bal_2019_df_clean.at[index, 'time_between_plays'] = 0  # or NaN
    elif row['Down'] > 1:
        if index - 1 in cle_bal_2019_df_clean.index:
            cle_bal_2019_df_clean.at[index, 'yardage_diff'] = abs(row['ToGo'] - cle_bal_2019_df_clean.at[index - 1, 'ToGo'])
        else:
            cle_bal_2019_df_clean.at[index, 'yardage_diff'] = 0  # or NaN
    elif row['Down'] == 1:
        cle_bal_2019_df_clean.at[index, 'yardage_diff'] = row['ToGo']
cle_bal_2019_df_clean.head()

In [118]:
def update_pass_columns(row):
    global previous_pass_tally  # Make previous_pass_tally accessible and modifiable within the function
    detail = row['Detail']
    if "pass incomplete" in detail:
        row['pass_complete'] = 0
        row['pass_incomplete'] = 1
        row['pass_play'] = 1
        row['pass_length'] = 0
    elif "pass complete" in detail:
        yards_match = re.search(r'for\s+(\d+)\s+yards', detail)
        if yards_match:
            yards = int(yards_match.group(1))
            row['pass_complete'] = 1
            row['pass_incomplete'] = 0
            row['pass_play'] = 1
            row['pass_length'] = yards
            previous_pass_tally += yards  # Update pass yards tally
        else:
            row['pass_complete'] = 1
            row['pass_incomplete'] = 0
            row['pass_play'] = 1
            row['pass_length'] = 0
    else:
        row['pass_complete'] = 0
        row['pass_incomplete'] = 0
        row['pass_play'] = 0
        row['pass_length'] = 0
    
    row['pass_yards_tally'] = previous_pass_tally  # Update pass yards tally
    return row

# Initialize cumulative passing yards
previous_pass_tally = 0

# Apply the function to each row to update pass columns
cle_off_df = cle_off_df.apply(update_pass_columns, axis=1)


In [119]:
def update_rush_columns(row):
    detail = row['Detail']
    match_words = ['middle for', 'tackle for', 'guard for', 'end for']

    for word in match_words:
        if word in detail:
            yards_match = re.search(r'for\s+(\d+)\s+yards', detail)
            if yards_match:
                row['rush_play'] = 1
                row['rush_length'] = int(yards_match.group(1))
            else:
                row['rush_play'] = 0
                row['rush_length'] = 0
            return row
    
    row['rush_play'] = 0
    row['rush_length'] = 0
    return row

def update_rush_yards_tally(row, previous_tally):
    if row['rush_play'] == 1:
        row['rush_yards_tally'] = previous_tally + row['rush_length']
    else:
        row['rush_yards_tally'] = previous_tally
    return row

# Apply the function to each row to update rush columns
# Apply the function to each row to update rush columns
cle_off_df = cle_off_df.apply(lambda row: update_rush_columns(row), axis=1)


# Calculate cumulative rushing yards
cumulative_rush_yards = 0
cle_off_df['rush_yards_tally'] = 0
for index, row in cle_off_df.iterrows():
    cle_off_df.loc[index] = update_rush_yards_tally(row, cumulative_rush_yards)
    cumulative_rush_yards = cle_off_df.loc[index, 'rush_yards_tally']

cle_off_df.head()


,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail,possession_team,yardage_diff_1_2,...,sacked,sack_yards,pass_complete,pass_incomplete,pass_play,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally
1,1,15:00,1,10,CLE 25,0,0,Baker Mayfield pass incomplete short middle in...,CLE,10.0,...,0,0,0,1,1,0,0,0,0,0
2,1,14:57,2,10,CLE 25,0,0,Nick Chubb left tackle for 4 yards (tackle by ...,CLE,0.0,...,0,0,0,0,0,0,0,1,4,4
3,1,14:15,3,6,CLE 29,0,0,Baker Mayfield sacked by Tyus Bowser for -5 yards,CLE,0.0,...,1,-5,0,0,0,0,0,0,0,4
9,1,11:21,1,10,CLE 16,0,0,Baker Mayfield pass complete short middle to J...,CLE,10.0,...,0,0,1,0,1,16,16,0,0,4
10,1,10:44,1,10,CLE 32,0,0,Baker Mayfield pass incomplete deep left inten...,CLE,10.0,...,0,0,0,1,1,0,16,0,0,4


In [120]:
def yards_to_promised_land(location):
    match = re.search(r'CLE\s+(\d+)', location)
    if match:
        yards = int(match.group(1))
        return 50 + yards
    else:
        return int(location.split()[1])


# Apply the function to the 'Detail' column and create a new column 'yards_to_endzone'
cle_off_df['yards_to_endzone'] = cle_off_df['Location'].apply(yards_to_promised_land)
cle_off_df.head()

,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail,possession_team,yardage_diff_1_2,...,sack_yards,pass_complete,pass_incomplete,pass_play,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally,yards_to_endzone
1,1,15:00,1,10,CLE 25,0,0,Baker Mayfield pass incomplete short middle in...,CLE,10.0,...,0,0,1,1,0,0,0,0,0,75
2,1,14:57,2,10,CLE 25,0,0,Nick Chubb left tackle for 4 yards (tackle by ...,CLE,0.0,...,0,0,0,0,0,0,1,4,4,75
3,1,14:15,3,6,CLE 29,0,0,Baker Mayfield sacked by Tyus Bowser for -5 yards,CLE,0.0,...,-5,0,0,0,0,0,0,0,4,79
9,1,11:21,1,10,CLE 16,0,0,Baker Mayfield pass complete short middle to J...,CLE,10.0,...,0,1,0,1,16,16,0,0,4,66
10,1,10:44,1,10,CLE 32,0,0,Baker Mayfield pass incomplete deep left inten...,CLE,10.0,...,0,0,1,1,0,16,0,0,4,82


create new column "conversion_successful" if  while on third down the resultmof the play is a rush length or pass length is greater than 'yards to go', then put 1 in the conversion seuccesfful 

In [121]:
def update_1d_conversion_successful(row):
    if row['Down'] == 1:
        gain = max(row['rush_length'], row['pass_length'])  # Calculate the gain based on rushing or passing
        if gain >= row['ToGo']:
            return 1
    return 0

# Apply the function to each row to update the conversion_successful column
cle_off_df['1D_conv_good'] = cle_off_df.apply(update_1d_conversion_successful, axis=1)
cle_off_df.head()


,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail,possession_team,yardage_diff_1_2,...,pass_complete,pass_incomplete,pass_play,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally,yards_to_endzone,1D_conv_good
1,1,15:00,1,10,CLE 25,0,0,Baker Mayfield pass incomplete short middle in...,CLE,10.0,...,0,1,1,0,0,0,0,0,75,0
2,1,14:57,2,10,CLE 25,0,0,Nick Chubb left tackle for 4 yards (tackle by ...,CLE,0.0,...,0,0,0,0,0,1,4,4,75,0
3,1,14:15,3,6,CLE 29,0,0,Baker Mayfield sacked by Tyus Bowser for -5 yards,CLE,0.0,...,0,0,0,0,0,0,0,4,79,0
9,1,11:21,1,10,CLE 16,0,0,Baker Mayfield pass complete short middle to J...,CLE,10.0,...,1,0,1,16,16,0,0,4,66,1
10,1,10:44,1,10,CLE 32,0,0,Baker Mayfield pass incomplete deep left inten...,CLE,10.0,...,0,1,1,0,16,0,0,4,82,0


In [122]:
def update_2d_conversion_successful(row):
    if row['Down'] == 2:
        gain = max(row['rush_length'], row['pass_length'])  # Calculate the gain based on rushing or passing
        if gain >= row['ToGo']:
            return 1
    return 0

# Apply the function to each row to update the conversion_successful column
cle_off_df['2D_conv_good'] = cle_off_df.apply(update_1d_conversion_successful, axis=1)
cle_off_df.head()

,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail,possession_team,yardage_diff_1_2,...,pass_incomplete,pass_play,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally,yards_to_endzone,1D_conv_good,2D_conv_good
1,1,15:00,1,10,CLE 25,0,0,Baker Mayfield pass incomplete short middle in...,CLE,10.0,...,1,1,0,0,0,0,0,75,0,0
2,1,14:57,2,10,CLE 25,0,0,Nick Chubb left tackle for 4 yards (tackle by ...,CLE,0.0,...,0,0,0,0,1,4,4,75,0,0
3,1,14:15,3,6,CLE 29,0,0,Baker Mayfield sacked by Tyus Bowser for -5 yards,CLE,0.0,...,0,0,0,0,0,0,4,79,0,0
9,1,11:21,1,10,CLE 16,0,0,Baker Mayfield pass complete short middle to J...,CLE,10.0,...,0,1,16,16,0,0,4,66,1,1
10,1,10:44,1,10,CLE 32,0,0,Baker Mayfield pass incomplete deep left inten...,CLE,10.0,...,1,1,0,16,0,0,4,82,0,0


In [123]:
def update_3d_conversion_successful(row):
    if row['Down'] == 3:  # Check if it's third down
        if row['rush_play'] == 1:  # If it's a rush play
            if row['rushing_yards'] >= row['ToGo']:  # Check if rushing yards are greater than or equal to yards to go
                return 1
        elif row['pass_play'] == 1:  # If it's a pass play
            if row['pass_length'] >= row['ToGo']:  # Check if pass length is greater than or equal to yards to go
                return 1
    return 0

# Apply the function to each row to update the conversion_successful column
cle_off_df['3D_conv_good'] = cle_off_df.apply(update_3d_conversion_successful, axis=1)
cle_off_df.head()


,Quarter,Time,Down,ToGo,Location,CLE,BAL,Detail,possession_team,yardage_diff_1_2,...,pass_play,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally,yards_to_endzone,1D_conv_good,2D_conv_good,3D_conv_good
1,1,15:00,1,10,CLE 25,0,0,Baker Mayfield pass incomplete short middle in...,CLE,10.0,...,1,0,0,0,0,0,75,0,0,0
2,1,14:57,2,10,CLE 25,0,0,Nick Chubb left tackle for 4 yards (tackle by ...,CLE,0.0,...,0,0,0,1,4,4,75,0,0,0
3,1,14:15,3,6,CLE 29,0,0,Baker Mayfield sacked by Tyus Bowser for -5 yards,CLE,0.0,...,0,0,0,0,0,4,79,0,0,0
9,1,11:21,1,10,CLE 16,0,0,Baker Mayfield pass complete short middle to J...,CLE,10.0,...,1,16,16,0,0,4,66,1,1,0
10,1,10:44,1,10,CLE 32,0,0,Baker Mayfield pass incomplete deep left inten...,CLE,10.0,...,1,0,16,0,0,4,82,0,0,0
